# Война и мир

> Поздравляем с освоением важных для анализа данных структур и функций Python! Пришло время применить их на реальном проекте. Сегодня мы обратимся к классике: применим анализ данных к «Войне и миру» Льва Толстого!

Текст произведения мы взяли в библиотеке [lib.ru](http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0073.shtml) и провели первоначальную обработку. Поскольку наша цель — обработка слов из этого произведения, мы разбили текст на слова и вывели каждое слово в отдельной строке. Кроме того, в местах, где начинаются главы, мы вывели строку `[new chapter]`.

[Скачать предобработанный файл](https://lms.skillfactory.ru/assets/courseware/v1/4c5a80c60b5a00504915158c4e17fe20/asset-v1:SkillFactory+DS-MASTERS+01SEPT2020+type@asset+block/war_peace_processed.txt)

Чтобы загрузить весь текст и разбить его на слова, в каждом задании используйте функцию `read_data`.

In [1]:
def read_data():
    data = open('data/war_peace_processed.txt', 'rt').read()
    return data.split('\n')

Функция вернёт массив всех слов по порядку, а в местах начала новых глав будет строка `[new chapter]`.

## Задание 1

> Входные данные: строка `target_word`.

Итак, начнём! Для начала посчитаем частоту использования отдельных слов в произведении. Для решения задачи воспользуемся уже знакомым нам словарём.

**Задание**. Напишите программу, которая переберет все слова и занесет их в словарь (назвать его можете как угодно). Увеличивайте счётчик при добавлении каждого нового слова, чтобы посчитать сколько раз это слово встречается в тексте.

**Подсказка**. Напоминаем, что метод `get` поможет проверить, какое значение соответствует ключу (слову) в словаре. Например, `words.get(word, 0)` вернёт либо значение из словаря, либо `0`, если соответствующее слово пока отсутствует.

В качестве результата выведите, сколько раз слово `target_word` было найдено в тексте. Например, для `target_word = 'князь'` ответ будет `1289`.

In [2]:
data = read_data()

In [3]:
word_count = {}
for w in data:
    c = word_count.get(w, 0)
    c += 1
    word_count[w] = c

In [4]:
target_word = 'князь'
word_count[target_word]

1289

## Задание 2

Входные данные: строка `target_word`.

Пришло время познакомиться с понятием *document frequency*.

*Document frequency* (для удобства сократим до *df*) — это доля документов, в которых встречается искомое слово. В нашем случае речь идёт не о документах, а о главах книги (выше мы писали, что в текстовом документе главы разделяются строкой `[new chapter]`).

**Задание**. Напишите программу, которая посчитает *document frequency* для заданного слова `target_word` и выведите результат на экран.

**Подсказка**. Вычислить *df* можно по формуле:

`number_of_documents_with_target_word / number_of_documents`

- `number_of_documents` — общее количество глав
- `number_of_documents_with_target_word` — количество глав, в которых встречается `target_word`

Объясним на примере: наш текст состоит из `171` главы (`number_of_documents`), а слово *человек* встречается в `115` главах (`number_of_documents_with_target_word`).

`df слова человек = 115 / 171 = 0.672514619883041`.

In [5]:
def calc_document_words(data):
    doc = [{}]
    word_count = [0]
    current_chapter = 0
    for w in data:
        chapter = doc[-1]
        if w == '[new chapter]':
            doc.append({})
            word_count.append(0)
            continue
        chapter[w] = chapter.get(w, 0) + 1
        word_count[-1] += 1
    return doc, word_count

doc, word_count = calc_document_words(data)

In [6]:
number_of_documents = len(doc)
number_of_documents

171

In [7]:
def calc_df(word):
    count = 0
    for chapter in doc:
        if chapter.get(word):
            count += 1
    return count / len(doc)

In [8]:
target_word = 'человек'
calc_df(target_word)

0.672514619883041

## Задание 3

Сделаем следущий шаг: посчитаем частоту употребления отдельного слова в документе (*term frequency*, или *tf*).

Проще всего объяснить, что такое *term frequency*, на примере:

`tf слова война = количество раз, когда слово война встречается в тексте главы / количество всех слов в тексте главы`

Попробуем посчитать частоту употребления слова *гостья* в 15-й главе (кстати, у нас главы нумеруются с 0). Слово гостья встречается в 15-й главе 10 раз, а общее количество слов — 1359.

tf слова гостья в 15 главе = 101359 = 0.007358351729212656.

**Задание**. Напишите программу, которая выведет частоту употребления заданного слова `target_word` в заданной главе `target_chapter`.

In [9]:
def calc_tf(word, chapter, word_count):
    return chapter.get(word, 0) / word_count

In [10]:
target_word = 'гостья'
target_chapter = 15

calc_tf(target_word, doc[target_chapter], word_count[target_chapter])

0.007358351729212656

## Задание 4

Входные данные:
- строка `target_word`
- число `target_chapter`

Пришло время дать разъяснения: для чего мы делали вычисления выше и что нас ждет впереди?

Если какое-то слово часто употребляется в документе, то, вероятно, этот документ что-то рассказывает о предмете/действии, описываемом этим словом. Скажем, если вы читаете книгу, в которой много раз употребляется слово *заяц*, то, вероятно, эта книга про зайцев.

Однако, если вы возьмёте слово *и*, то оно будет встречаться почти в каждой книге много раз. Таким образом, если мы хотим найти наиболее значимые слова в книге, мы, с одной стороны, хотим найти наиболее частые слова, а с другой — убрать те, которые не несут важной информации, так как встречаются везде.

**Подсказка**. Такая задача хорошо решается с помощью `tf*idf` — статистической метрики для оценки важности слова в тексте. Другими словами, `tf*idf` — это «контрастность» слова в документе (насколько оно выделяется среди других слов).

- `tf*idf = term frequency * inverse document frequency` 
- `tf` — это частотность термина, которая измеряет, насколько часто термин встречается в документе.
- `idf` — это обратная документная частотность термина. Она измеряет непосредственно важность термина во всём множестве документов.

Чтобы получить `idf`, необходимо поделить 1 на полученную в Задании 2 документную частоту (*df*).

Мы будем использовать не сырые значения *tf* и *idf*, а их логарифмы, то есть `log(1+tf) * log(idf)`. Сейчас мы не будем заострять внимания на том, почему следует использовать именно логарифмы — это долгий разговор. Поговорим об этом в курсе *Математика и алгоритмы для машинного обучения*.

В качестве примера измерим *tf*idf* слова *анна* в главе 4. Слово *анна* встречается в указанной главе 7 раз (`tf`), при этом в 4 главе 1060 слов (`chapter_size`), всего же слово *анна* упоминается в 32 главах (*df*) из 171 (`chapter_count`).

Таким образом, *tf*idf* данного слова в данной главе будет равно `math.log(1 + tf / chapter_size) * math.log(1 / df)`, то есть
0.011031063403921838.

Задание. Напишите программу, которая выведет значение tf*idf для заданного слова target_word в заданной главе target_chapter.

In [11]:
import math

def calc_tf_idf(word, chapter, word_count):
    tf = calc_tf(word, chapter, word_count)
    df = calc_df(word)
    return math.log(1 + tf) * math.log(1 / df)

In [12]:
target_word = 'анна'
target_chapter = 4
calc_tf_idf(target_word, doc[target_chapter], word_count[target_chapter])

0.011031063403921838

## Задание 5

Теперь, когда мы умеем вычислять *tf*idf* для каждого слова в главе, мы можем найти те слова, которые являются самыми «контрастными» для данной главы, то есть они могут являться в своём роде заголовком для главы.

**Задача**: напишите код, который выведет на экран через пробел три слова, имеющие самое высокое значение *tf*idf* в заданной главе `target_chapter` в порядке убывания *tf*idf*.

Например, для 4 главы ответом будет: `павловна анна прядильной`

In [13]:
target_chapter = 4
chapter = doc[target_chapter]
wc = word_count[target_chapter]
words = chapter.keys()
rated_words = [
    (w, calc_tf_idf(w, chapter, wc))
    for w in words
]
rated_words = sorted(rated_words, key=lambda x: x[1], reverse=True)
top_words = rated_words[:3]
result = ' '.join([n for n, r in top_words])
result # тетушку == прядильной ¯\_(ツ)_/¯

'павловна анна тетушку'